In [2]:
import numpy as np
import torch

# PyTorchテンソルのprint表示設定を変更
# edgeitems: 各次元の先頭・末尾に表示する要素数（中間は "..." で省略される）
# threshold: 表示の総要素数がこれを超えると省略表示になる（全体で50要素以下ならすべて表示）
torch.set_printoptions(edgeitems=2, threshold=50)

In [5]:
import imageio

img_arr = imageio.imread("../../data/p1ch4/image-dog/bobby.jpg")

img_arr.shape

/var/folders/0t/6d51btm11zz6lw61fgrmf7z00000gn/T/ipykernel_87508/2817573562.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread("../../data/p1ch4/image-dog/bobby.jpg")


(720, 1280, 3)

In [6]:
# NumPy配列（H, W, C）をPyTorchテンソル（C, H, W）に変換する処理

# NumPy配列をPyTorchテンソルに変換（共有メモリ、コピーはされない）
img = torch.from_numpy(img_arr)  # img_arr: shape = (高さ, 幅, チャンネル)

# 次元を並び替えて、PyTorchの標準形式である (C, H, W) に変換
out = img.permute(2, 0, 1)  # 元の (H, W, C) → (C, H, W)

In [7]:
# バッチサイズ3、RGB画像（3チャネル）、サイズ256x256の空バッチを作成
# dtype=torch.uint8 は画像のピクセル値（0〜255）を格納する形式に適している

batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)
# shape: [3, 3, 256, 256] → 3枚の画像、各画像は [C, H, W] = [3, 256, 256]

In [8]:
import os
import imageio
import torch

# データディレクトリ（画像ファイルが保存されているフォルダ）
data_dir = "../../data/p1ch4/image-cats/"

# ディレクトリ内のPNG画像ファイル名だけを抽出
filenames = [
    name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == ".png"
]

# ファイル数に応じてバッチテンソルを初期化（ここでは事前に batch が定義済みとする）
# 例: batch = torch.zeros(len(filenames), 3, 256, 256, dtype=torch.uint8)

# 各画像を読み込み、テンソル化してバッチに格納
for i, filename in enumerate(filenames):
    # 画像ファイルを読み込み → NumPy配列 (H, W, C) 型
    img_arr = imageio.imread(os.path.join(data_dir, filename))

    # NumPy配列 → PyTorchテンソルに変換（uint8型のまま）
    img_t = torch.from_numpy(img_arr)

    # 次元順を変更：（H, W, C）→（C, H, W）に並べ替え
    img_t = img_t.permute(2, 0, 1)

    # 万が一アルファチャンネルがある場合は、RGBの3チャンネルだけに制限 <1>
    img_t = img_t[:3]

    # バッチテンソルにコピー（i番目の画像を格納）
    batch[i] = img_t

/var/folders/0t/6d51btm11zz6lw61fgrmf7z00000gn/T/ipykernel_87508/944037454.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread(os.path.join(data_dir, filename))


In [9]:
batch = batch.float()
batch /= 255.0

In [10]:
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std